In [1]:
import tensorflow as tf
from tensorflow import keras

Install and import the Keras Tuner.

In [2]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 71kB 6.1MB/s 


In [3]:
import kerastuner as kt

## Download and prepare the dataset


In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## Define the model


In [6]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

## Instantiate the tuner and perform hypertuning



In [7]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [8]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [9]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 19s]
val_accuracy: 0.8575833439826965

Best val_accuracy So Far: 0.8866666555404663
Total elapsed time: 00h 05m 35s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 160 and the optimal learning rate for the optimizer
is 0.001.



## Train the model


In [10]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.6408 - accuracy: 0.7740 - val_loss: 0.4124 - val_accuracy: 0.8525
Epoch 2/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3942 - accuracy: 0.8590 - val_loss: 0.3860 - val_accuracy: 0.8582
Epoch 3/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3461 - accuracy: 0.8736 - val_loss: 0.3552 - val_accuracy: 0.8708
Epoch 4/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3163 - accuracy: 0.8851 - val_loss: 0.3847 - val_accuracy: 0.8552
Epoch 5/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2987 - accuracy: 0.8908 - val_loss: 0.3295 - val_accuracy: 0.8794
Epoch 6/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2816 - accuracy: 0.8956 - val_loss: 0.3299 - val_accuracy: 0.8833
Epoch 7/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2678 - accuracy: 0.8987 - val_loss: 0.3068 - val_accuracy:

In [11]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_test, label_test, epochs=best_epoch)

Epoch 1/36
313/313 [==============================] - 1s 2ms/step - loss: 0.9338 - accuracy: 0.6670
Epoch 2/36
313/313 [==============================] - 1s 2ms/step - loss: 0.5275 - accuracy: 0.8131
Epoch 3/36
313/313 [==============================] - 1s 2ms/step - loss: 0.4669 - accuracy: 0.8357
Epoch 4/36
313/313 [==============================] - 1s 2ms/step - loss: 0.4266 - accuracy: 0.8442
Epoch 5/36
313/313 [==============================] - 1s 2ms/step - loss: 0.3907 - accuracy: 0.8586
Epoch 6/36
313/313 [==============================] - 1s 2ms/step - loss: 0.3768 - accuracy: 0.8605
Epoch 7/36
313/313 [==============================] - 1s 2ms/step - loss: 0.3428 - accuracy: 0.8773
Epoch 8/36
313/313 [==============================] - 1s 2ms/step - loss: 0.3511 - accuracy: 0.8729
Epoch 9/36
313/313 [==============================] - 1s 2ms/step - loss: 0.3163 - accuracy: 0.8874
Epoch 10/36
313/313 [==============================] - 1s 2ms/step - loss: 0.3170 - accuracy: 0.8835

In [12]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 1ms/step - loss: 0.0953 - accuracy: 0.9661
[test loss, test accuracy]: [0.09534787386655807, 0.9660999774932861]
